In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
from environment import ImageBanditEnv, Experiment, ImageBanditEnv2
from algorithms import LinUCB, CNN_UCB, ViT_UCB
from pathlib import Path

# Environment Setup
Change `exp_type` to run different experiment

In [ ]:
exp_type = "anime" # ="mnist" or "anime2"
if exp_type=="anime":
    f = "anime_env.pkl"
    if Path(f).is_file():
        environment = ImageBanditEnv.load(f)
    else:
        environment = ImageBanditEnv("data.jsonl", 3, 1)
        environment.save(f)
elif exp_type=="mnist":
    f = "mnist_env.pkl"
    if Path(f).is_file():
        environment = ImageBanditEnv.load(f)
    else:
        environment = ImageBanditEnv.from_mnist(Path("mnist dataset"), 3)
        environment.save(f)
elif exp_type=="anime2":
    f = "anime2_env.pkl"
    if Path(f).is_file():
        environment = ImageBanditEnv2.load(f)
    else:
        environment = ImageBanditEnv2("data.jsonl", 3, 1)
        environment.save(f)
else:
    print("Unknown experiment type")

Values are copied from hyperparameter tuning

⚠️ The feature covariance matrices can go up to $20000\times20000$, which takes up a few GB

In [ ]:
detailed = exp_type!="anime2"
linucb = LinUCB(environment, alpha=84, detailed=detailed)
cnnucb = CNN_UCB(environment, alpha=75, lambda_reg=2.7, detailed=detailed)
vitucb = ViT_UCB(environment, lora_r=20, alpha=85, lambda_reg=4, lora_alpha=24, detailed=detailed)
experiment = Experiment(environment, [linucb, cnnucb, vitucb], results=f"{exp_type} results", detailed=detailed)

# Run experiment
Experiments are ran for 10000 time steps. With an NVIDIA RTX A4000, LinUCB takes 1-2 hours to run. CNNUCB and ViTUCB takes 12-15 hours to run

In [ ]:
experiment.run(10000)

In [ ]:
experiment.generate_results(100)

In [ ]:
import pickle
with open(f'{exp_type}_exp.pkl', 'wb') as f:
    pickle.dump(experiment, f)